In [3]:
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types import XraiAttribution
from google.cloud.aiplatform_v1.types.explanation import ExplanationParameters

project_id = "udemy-mlops"
region = "us-central1"
staging_bucket="gs://sid-kubeflow-v1"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

<h4>Custom Model Training

In [ ]:
job = aiplatform.CustomTrainingJob(
        display_name="coupoon-recommendation-training",
        script_path="model-training-code.py",
        container_uri = "us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest",
        requirements=["gcsfs","category_encoders==2.6.1","imbalanced-learn==0.11.0", "scikit-learn>=0.24.0"]
    )

job.run(
    replica_count=1,
    machine_type="n1-standard-4",
    sync=True
)
job.wait()

<h4>Upload Trained Model to Model Registry 

In [ ]:
display_name = "coupon-recommendation-sdk"

artifact_uri = "gs://sid-kubeflow-v1/coupon-recommendation/artifacts/"

serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest"

exp_metadata = {"inputs": {"input_features": {}}, "outputs": {"predicted_outcome": {}}}

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        explanation_metadata=exp_metadata,
        explanation_parameters=ExplanationParameters(
                sampled_shapley_attribution=XraiAttribution()
            ),
        sync=False
    )

model.wait()

<h4>Deploy Model to Vertex Endpoint

In [ ]:
deployed_model_display_name = "coupon-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

<h4> Run Online Predictions

In [ ]:
INSTANCE =[0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  2,
  3,
  0,
  0,
  3,
  2,
  1,
  0,
  0]
instances_list = [INSTANCE]
prediction = endpoint.predict(instances_list)
print(prediction)

<h4> Define the input features used for model training

In [21]:
features = ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14',
       'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21',
       'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'time', 'expiration',
       'age', 'education', 'income', 'Bar', 'CoffeeHouse', 'CarryAway',
       'Restaurant20To50', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min',
       'direction_same']

<h4> Run explainability

In [ ]:
response = endpoint.explain(instances=instances_list)

for explanation in response.explanations:
    attributions = explanation.attributions
    for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                # print("   output_index:", output_index)
                attrs = attribution.feature_attributions
                rows = {"feature_name": [], "attribution": []}
                for i, val in enumerate(features):
                    rows["feature_name"].append(val)
                    rows["attribution"].append(attrs["input_features"][i])
            
            print(rows)
            print("-------------------------------------")

<h4> Run explainability for batch predictions

In [ ]:
# model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/6594037313586593792')

gcs_input_uri = 'gs://sid-kubeflow-v1/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://sid-kubeflow-v1/coupon-recommendation/batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    generate_explanation=True,
    sync=False
)